In [1]:
from transformer_lens import HookedTransformer
from sae_lens import SAE
import plotly.express as px
import torch
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
device = "cuda"
model = HookedTransformer.from_pretrained_no_processing("meta-llama/Meta-Llama-3-8B", device = device)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B into HookedTransformer


In [4]:
from sae_lens import SAE
# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience. 
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "sae-llama-3-8b-eai", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = "blocks.3.hook_resid_post", # won't always be a hook point
    device = "cuda",
)

layers.3/cfg.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

In [5]:
sae.cfg.context_size = 128

In [6]:
_, cache = model.run_with_cache("I like to eat apples and bananas", names_filter = [sae.cfg.hook_name])
feature_acts = sae.encode(cache[sae.cfg.hook_name])

In [7]:
from sae_lens import ActivationsStore

activation_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    streaming=True,
    # fairly conservative parameters here so can use same for larger
    # models without running out of memory.
    store_batch_size_prompts=8,
    train_batch_size_tokens=4096,
    n_batches_in_buffer=4,
    device=device,
)


In [8]:
from sae_lens import run_evals

metrics = run_evals(sae, activation_store, model, n_eval_batches=8, eval_batch_size_prompts=8)
print(metrics)

{'metrics/l2_norm': 2.7873101234436035, 'metrics/l2_ratio': 0.893515408039093, 'metrics/l2_norm_in': 5.343474388122559, 'metrics/CE_loss_score': -0.0256333872966934, 'metrics/ce_loss_without_sae': 2.8757602870464325, 'metrics/ce_loss_with_sae': 11.989570021629333, 'metrics/ce_loss_with_ablation': 11.761783599853516}


In [12]:
import circuitsvis as cv 
prompt = "1 2 3 4 5 6 7 8 9 10 11 12"
logits, cache = model.run_with_cache(prompt)
display(cv.logits.token_log_probs(model.to_tokens(prompt), model(prompt)[0].log_softmax(dim=-1), model.to_string))


## Compare Activations

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
torch.set_grad_enabled(False)  # Disable gradient calculation for inference

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_name = "meta-llama/Meta-Llama-3-8B"

#### HuggingFace Version

In [4]:
hf_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hf_model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [5]:
tokenizer.eos_token_id

128001

In [6]:
input = torch.tensor(tokenizer.encode("Let's try something quite different. The problem we have is that")).unsqueeze(0).to(device)
print(f"Input: {input}")

layers=[1, 4, 7] # offset by one so we get the output of 0, 3, and 6
all_hidden_states = [[] for _ in layers]

with torch.no_grad():
    outputs = hf_model(
        input,
        output_hidden_states=True
        )

    hidden_list = outputs.hidden_states[:-1]  # Exclude the final layer

    # Select only the specified layers
    hidden_list = [hidden_list[i] for i in layers]

# Store the hidden states
for i, hidden in enumerate(hidden_list):
    all_hidden_states[i].append(hidden.cpu())

# Concatenate all batches for each layer
all_hidden_states = [torch.cat(states, dim=0) for states in all_hidden_states]

print(f"Output: {outputs.logits.shape}")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Input: tensor([[128000,  10267,    596,   1456,   2555,   5115,   2204,     13,    578,
           3575,    584,    617,    374,    430]], device='cuda:0')
Output: torch.Size([1, 14, 128256])


#### TransformerLens Version

In [7]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained_no_processing(model_name, device="cuda", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B into HookedTransformer


In [8]:
logits, cache = model.run_with_cache(input, prepend_bos=False)
cache['blocks.3.hook_resid_post'].shape

torch.Size([1, 14, 4096])

#### Comparison

In [12]:
assert cache['blocks.3.hook_resid_post'].shape == all_hidden_states[1].shape
activation_match = torch.allclose(cache['blocks.6.hook_resid_post'].cpu(), all_hidden_states[2], atol=1e-6)
# print samples of each
print(f"Activation match: {activation_match}\n")

print("\nHuggingFaceExtractor:")
print(all_hidden_states[2][0, 5, :10])

print("HookedTransformer:")
print(cache['blocks.6.hook_resid_post'][0, 5, :10])

print(f"\nMean value of HF activations: {all_hidden_states[2].mean()}")
print(f"Mean value of TL activations: {cache['blocks.6.hook_resid_post'].mean()}")


Activation match: False


HuggingFaceExtractor:
tensor([ 0.0596,  0.0595, -0.0433, -0.0446, -0.1433,  0.0310,  0.0668, -0.0174,
        -0.0274,  0.0760])
HookedTransformer:
tensor([ 0.0504,  0.0605, -0.0522, -0.0435, -0.1374,  0.0271,  0.0637, -0.0272,
        -0.0220,  0.0875], device='cuda:0')

Mean value of HF activations: -0.004082582890987396
Mean value of TL activations: -0.004054217599332333


In [15]:
import torch
import numpy as np

def compare_hidden_states(state1, state2, name1="HookedTransformer", name2="HiddenStateExtractor", rtol=1e-5, atol=1e-2):
    """
    Compare two hidden states and return detailed information about their differences.
    
    Args:
    state1, state2: The hidden states to compare (torch.Tensor)
    name1, name2: Names of the sources of the hidden states (str)
    rtol: Relative tolerance for comparison
    atol: Absolute tolerance for comparison
    
    Returns:
    A dictionary containing information about the comparison
    """
    if state1.shape != state2.shape:
        return {
            "match": False,
            "reason": "Shapes do not match",
            f"{name1}_shape": state1.shape,
            f"{name2}_shape": state2.shape
        }
    
    # Convert to numpy for easier handling
    array1 = state1.detach().cpu().numpy()
    array2 = state2.detach().cpu().numpy()
    
    # Check if the arrays are close
    is_close = np.isclose(array1, array2, rtol=rtol, atol=atol)
    match = np.all(is_close)
    
    if match:
        return {"match": True}
    
    # Find where values are not close
    not_close = ~is_close
    diff_indices = np.where(not_close)
    
    # Prepare a list of differences
    differences = []
    for index in zip(*diff_indices):
        value1 = array1[index]
        value2 = array2[index]
        abs_diff = abs(value1 - value2)
        rel_diff = abs_diff / max(abs(value1), abs(value2))
        differences.append({
            "index": index,
            "value1": value1,
            "value2": value2,
            "absolute_difference": abs_diff,
            "relative_difference": rel_diff
        })
    
    # Sort differences by absolute difference
    differences.sort(key=lambda x: x["absolute_difference"], reverse=True)
    
    # Compute summary statistics
    abs_diffs = np.abs(array1 - array2)
    
    return {
        "match": False,
        "num_different_elements": np.sum(not_close),
        "total_elements": array1.size,
        "max_absolute_difference": np.max(abs_diffs),
        "mean_absolute_difference": np.mean(abs_diffs),
        "median_absolute_difference": np.median(abs_diffs),
        "top_10_differences": differences[:10]
    }

# Example usage
def main():
    # Assuming you have your hidden states in these variables
    hooked_transformer_state = cache['blocks.6.hook_resid_post'].cpu()
    hidden_state_extractor_state = all_hidden_states[2]

    comparison_result = compare_hidden_states(hooked_transformer_state, hidden_state_extractor_state)

    print(f"Hidden states match: {comparison_result['match']}")
    if not comparison_result['match']:
        print(f"Number of different elements: {comparison_result['num_different_elements']} out of {comparison_result['total_elements']}")
        print(f"Maximum absolute difference: {comparison_result['max_absolute_difference']}")
        print(f"Mean absolute difference: {comparison_result['mean_absolute_difference']}")
        print(f"Median absolute difference: {comparison_result['median_absolute_difference']}")
        print("\nTop 10 largest differences:")
        for diff in comparison_result['top_10_differences']:
            print(f"At index {diff['index']}:")
            print(f"  HookedTransformer value: {diff['value1']}")
            print(f"  HiddenStateExtractor value: {diff['value2']}")
            print(f"  Absolute difference: {diff['absolute_difference']}")
            print(f"  Relative difference: {diff['relative_difference']}")
            print()

if __name__ == "__main__":
    main()

Hidden states match: False
Number of different elements: 22718 out of 57344
Maximum absolute difference: 0.13030779361724854
Mean absolute difference: 0.010224463418126106
Median absolute difference: 0.00719045102596283

Top 10 largest differences:
At index (0, 6, 4055):
  HookedTransformer value: 0.5900180339813232
  HiddenStateExtractor value: 0.7203258275985718
  Absolute difference: 0.13030779361724854
  Relative difference: 0.1809011846780777

At index (0, 8, 4055):
  HookedTransformer value: 1.271518588066101
  HiddenStateExtractor value: 1.1430875062942505
  Absolute difference: 0.12843108177185059
  Relative difference: 0.10100606083869934

At index (0, 13, 103):
  HookedTransformer value: 0.285245805978775
  HiddenStateExtractor value: 0.18846748769283295
  Absolute difference: 0.09677831828594208
  Relative difference: 0.3392804265022278

At index (0, 7, 1800):
  HookedTransformer value: 0.020996078848838806
  HiddenStateExtractor value: -0.0670798271894455
  Absolute differe

#### Test Generation

In [7]:
model.generate("Once upon a time, in a galaxy far, far away", max_new_tokens=50, temperature=0)

  0%|          | 0/50 [00:00<?, ?it/s]

'Once upon a time, in a galaxy far, far away, there was a little boy who loved to play with his toys. One day, he decided to play a game with his toys. He would hide them in different places around the house, and then he would go on a treasure hunt to find them'

In [12]:
def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7, top_p=0.9):
    """
    Generate text using a specified Hugging Face model.

    Args:
    model_name (str): The name of the Hugging Face model to use.
    prompt (str): The initial text to start the generation.
    max_length (int): The maximum length of the generated text (including the prompt).
    temperature (float): Controls randomness in generation. Higher values increase randomness.
    top_p (float): Controls diversity of generation. Lower values make output more focused.

    Returns:
    str: The generated text.
    """

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Encode the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

# Example usage
if __name__ == "__main__":
    #model_name = "gpt2"  # You can change this to any other Hugging Face model
    prompt = "Once upon a time, in a galaxy far, far away"

    generated_text = generate_text(hf_model, tokenizer, prompt)
    print(f"Generated text:\n{generated_text}")

Generated text:
Once upon a time, in a galaxy far, far away, there was a little girl who loved to read. She loved to read so much that she would read anything she could get her hands on. She would read books, magazines, newspapers, cereal boxes, and even the back of the shampoo bottle. She would read so much that she would often forget to eat or sleep. Her parents were worried about her, but they knew that reading was her passion, and they supported her in her
